In [ ]:
import pandas as pd
pd.set_option('display.max_columns',None)

X_test = pd.read_csv('data/X_test.csv')
X_train = pd.read_csv('data/X_train.csv')
y_train = pd.read_csv('data/y_train.csv')

# EDA 및 데이터 전처리

In [ ]:
train = pd.merge(X_train,y_train)
test = X_test.copy()

#1-1. 결측값 확인
#print(train.isnull().sum())
#print(test.isnull().sum())
#환불금액 null 값들 0으로 교체해야 한다는 것 확인
train['환불금액'] = train['환불금액'].fillna(0)
test['환불금액'] = test['환불금액'].fillna(0)

#1-2. 기술통계 확인
#print(train.describe())
#print(test.describe())
#총 구매액과 최대구매액에서 최소값이 음수로 나오는 것 확인. 0 이상인 값들만 취급하기로 함

train = train[(train['총구매액'] > 0) & (train['최대구매액'] > 0)]
test = test[(test['총구매액'] > 0) & (test['최대구매액'] > 0)]

#인덱스 재정렬
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)

#데이터 타입 확인
#print(train.info())
#주구매지점, 주구매상품 제외하고 전부 수치형.
#범주형은 원핫인코딩, 수치형은 표준화 스케일링 위해 분리

#수치형 분리방법
train_numer = train.select_dtypes(exclude = 'object')
test_numer = test.select_dtypes(exclude = 'object')
train_category = train.select_dtypes(include = 'object')
test_category = test.select_dtypes(include = 'object')

num_columns = train_numer.columns[1:8] #cust_id, gender(종속변수) 제외
cat_columns = train_category.columns

#수치형 스케일링
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_numer.iloc[:,1:8] = pd.DataFrame(sc.fit_transform(train_numer.iloc[:,1:8]))
test_numer.iloc[:,1:] = pd.DataFrame(sc.transform(test_numer.iloc[:,1:])) #cust_id 제외

#범주형 원핫인코딩
oh_product_train = pd.get_dummies(train_category['주구매상품'])
oh_store_train = pd.get_dummies(train_category['주구매지점'])
oh_product_test = pd.get_dummies(test_category['주구매상품'])
oh_store_test = pd.get_dummies(test_category['주구매지점'])

#스케일링, 원핫인코딩 된 데이터 프레임 생성
train = pd.concat([train_numer,oh_product_train,oh_store_train],axis=1)
test = pd.concat([test_numer,oh_product_test,oh_store_test],axis=1)

# 모델링 및 구현

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

X = train.drop(['cust_id','gender','소형가전'],axis = 1) #train에만 소형가전이 있고 test에는 없어서 드랍
y = train['gender']
X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size = 0.25, stratify = y)

lr = LogisticRegression()
lr.fit(X,y)
scores = cross_val_score(lr,X,y,cv = 5, scoring = 'roc_auc')

print("scores : ", scores)
print("\nAverage Score : ", scores.mean())

test_gender = test.drop(['cust_id'],axis = 1)
test_gender = pd.Series(lr.predict_proba(test_gender)[:,1], name = 'gender')
results = pd.concat([test.cust_id, test_gender],axis=1)
#print(results)
results.to_csv("수험번호.csv",index = False)